In [113]:
import csv
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

from sqlalchemy import create_engine, inspect, func, MetaData
from sqlalchemy import Column, Integer, String, Float, Date

from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [87]:
clean_measurements = "resources/clean_measurements.csv"
clean_measurements_df = pd.read_csv(clean_measurements)
clean_measurements_df.head()

,Unnamed: 0,station,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,1,USC00519397,2010-01-02,0.00,63
2,2,USC00519397,2010-01-03,0.00,74
3,3,USC00519397,2010-01-04,0.00,76
4,5,USC00519397,2010-01-07,0.06,70


In [88]:
clean_stations = "resources/clean_stations.csv"
clean_stations_df = pd.read_csv(clean_stations)
clean_stations_df.head()

,Unnamed: 0,station,name,latitude,longitude,elevation
0,0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [89]:
class Measurements(Base):
    __tablename__ = 'measurements'
    id = Column(Integer, primary_key=True)
    station = Column(String(15))
    date = Column(String(15))
    prcp = Column(Float)
    tobs = Column(Integer)

In [90]:
class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    station = Column(String(15))
    name = Column(String(45))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

In [91]:
Base.metadata.tables

immutabledict({'measurements': Table('measurements', MetaData(bind=None), Column('id', Integer(), table=<measurements>, primary_key=True, nullable=False), Column('station', String(length=15), table=<measurements>), Column('date', String(length=15), table=<measurements>), Column('prcp', Float(), table=<measurements>), Column('tobs', Integer(), table=<measurements>), schema=None), 'station': Table('station', MetaData(bind=None), Column('id', Integer(), table=<station>, primary_key=True, nullable=False), Column('station', String(length=15), table=<station>), Column('name', String(length=45), table=<station>), Column('latitude', Float(), table=<station>), Column('longitude', Float(), table=<station>), Column('elevation', Float(), table=<station>), schema=None)})

In [92]:
cleaned_measurements_df = pd.read_csv('Resources/clean_measurements.csv')
cleaned_stations_df = pd.read_csv('Resources/clean_stations.csv')

In [93]:
engine = create_engine("sqlite:///hawaii.sqlite")
conn = engine.connect()

Base.metadata.create_all(engine)

In [94]:
session = Session(bind=engine)

In [97]:
Base = automap_base()

In [98]:
Base.prepare(engine, reflect = True)

In [99]:
Base.classes.keys()

['measurements', 'measurements_table', 'station']

In [100]:
Measurements = Base.classes.measurements

In [122]:
data_measurements = cleaned_measurements_df.to_dict(orient='records')

In [123]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [124]:
measurement_table = sqlalchemy.Table('measurements', metadata, autoload=True)

In [126]:
conn.execute(measurement_table.insert(), data_measurements)

In [127]:
# testing
conn.execute("select * from measurements limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-07', 0.06, 70)]

In [129]:
station_data = cleaned_stations_df.to_dict(orient='records')

In [130]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [132]:
station_table = sqlalchemy.Table('station', metadata, autoload=True)

In [133]:
conn.execute(station_table.insert(), station_data)

In [134]:
# testing
conn.execute("select * from station limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]